In [21]:
import pandas as pd
import sqlite3
import os
import matplotlib.pyplot as plt

# ==========================================
# 1. CONFIGURATION & PATHS
# ==========================================
RAW_DATA_PATH = r"C:\Users\Jesus Sanchez\Desktop\ALEXIS\1. Pre-Trabajo\1. Supply Chain Intelligence\data\raw\online_retail_II.xlsx"
PROCESSED_DIR = r"C:\Users\Jesus Sanchez\Desktop\ALEXIS\1. Pre-Trabajo\1. Supply Chain Intelligence\data\processed"

# Create directory if it doesn't exist
os.makedirs(PROCESSED_DIR, exist_ok=True)

# ==========================================
# 2. DATA INGESTION
# ==========================================
# Load multiple sheets and consolidate into a single DataFrame
print("⏳ Loading data from Excel sheets...")
df_dict = pd.read_excel(RAW_DATA_PATH, sheet_name=None)

consolidated_frames = []
for sheet_name, frame in df_dict.items():
    frame['Period'] = sheet_name
    consolidated_frames.append(frame)

df = pd.concat(consolidated_frames, ignore_index=True)

print(f"✅ Total records loaded: {df.shape[0]:,}")
print(f"✅ Columns identified: {df.columns.tolist()}")

# ==========================================
# 3. DATA INSPECTION & EXPLORATORY ANALYSIS
# ==========================================
print("\n--- Column Summary & Null Values ---")
print(df.info())

# Detailed statistical summary with formatting
target_cols = ["Quantity", "Price"]
custom_percentiles = [.01, .05, .25, .5, .75, .95, .99]

stats_summary = df[target_cols].describe(percentiles=custom_percentiles)
formatted_summary = stats_summary.style.format({
    "Quantity": "{:,.2f}",
    "Price": "£{:,.2f}"
})

print("\n--- Statistical Overview (Raw Data) ---")
display(formatted_summary)

# ==========================================
# 4. DATA CLEANSING & OUTLIER REMOVAL
# ==========================================
# Standardize column types
df['Invoice'] = df['Invoice'].astype(str)

# --- A. Segment Cancellations ---
# Invoices starting with 'C' indicate cancelled orders
df_cancellations = df[df['Invoice'].str.startswith('C')].copy()

# --- B. Clean Main Dataset ---
# 1. Remove rows without descriptions
df_main_clean = df.dropna(subset=['Description']).copy()

# 2. Exclude cancellations from the main transactional dataset
df_main_clean = df_main_clean[~df_main_clean['Invoice'].str.startswith('C')]

# 3. Filter for valid transactions (Positive values only)
df_main_clean = df_main_clean[(df_main_clean['Quantity'] > 0) & (df_main_clean['Price'] > 0)]

# 4. Outlier Removal via Inter-percentile Range (1% - 99%)
q_low, q_high = df_main_clean['Quantity'].quantile([0.01, 0.99])
p_low, p_high = df_main_clean['Price'].quantile([0.01, 0.99])

df_main_clean = df_main_clean[
    (df_main_clean['Quantity'] >= q_low) & (df_main_clean['Quantity'] <= q_high) &
    (df_main_clean['Price'] >= p_low) & (df_main_clean['Price'] <= p_high)
]

print(f"\n✅ Main Clean Dataset: {len(df_main_clean):,} records")
print(f"✅ Cancellations Dataset: {len(df_cancellations):,} records")
print(f"📊 Applied Limits: Quantity [{q_low}-{q_high}], Price [{p_low}-{p_high}]")

# ==========================================
# 5. DATA EXPORT (CSV & SQLite)
# ==========================================
# CSV Exports
main_csv_path = os.path.join(PROCESSED_DIR, "cleaned_retail_data.csv")
canc_csv_path = os.path.join(PROCESSED_DIR, "cancellation_retail_data.csv")

df_main_clean.to_csv(main_csv_path, index=False)
df_cancellations.to_csv(canc_csv_path, index=False)

# SQLite Export
db_path = os.path.join(PROCESSED_DIR, "retail_vault.db")
conn = sqlite3.connect(db_path)

print("\n📦 Exporting to SQLite database...")
df_main_clean.to_sql('transactions', conn, if_exists='replace', index=False)
df_cancellations.to_sql('cancellations', conn, if_exists='replace', index=False)

conn.close()
print(f"✅ Database 'retail_vault.db' successfully updated at: {PROCESSED_DIR}")



⏳ Loading data from Excel sheets...
✅ Total records loaded: 1,067,371
✅ Columns identified: ['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country', 'Period']

--- Column Summary & Null Values ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 9 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
 8   Period       1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 73.3+ MB
Non

,Quantity,Price
count,"1,067,371.00","£1,067,371.00"
mean,9.94,£4.65
std,172.71,£123.55
min,"-80,995.00","£-53,594.36"
1%,-3.00,£0.21
5%,1.00,£0.42
25%,1.00,£1.25
50%,3.00,£2.10
75%,10.00,£4.15
95%,30.00,£9.95



✅ Main Clean Dataset: 1,014,932 records
✅ Cancellations Dataset: 19,494 records
📊 Applied Limits: Quantity [1.0-100.0], Price [0.29-18.0]

📦 Exporting to SQLite database...
✅ Database 'retail_vault.db' successfully updated at: C:\Users\Jesus Sanchez\Desktop\ALEXIS\1. Pre-Trabajo\1. Supply Chain Intelligence\data\processed
